In [3]:
#!pip install neo4j
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
import graph_model as gm
from sklearn.metrics import mean_squared_error
from math import sqrt

np.set_printoptions(suppress=True)

In [88]:
uri = "bolt://3.220.233.169:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "i-0e23d19f0d8795714"))

In [9]:
def cypher(driver, query, results_columns):
    """This is wrapper for sending basic cypher queries to a neo4j server. Input is a neo4j connection
    driver, a string representing a cypher queryand a list of string for data frame column names.
    returns the dataframe of the results."""

    with driver.session() as session:
        result = session.run(query)

    result_df = pd.DataFrame(result.values(), columns=results_columns)

    return result_df


def expected_rating(rating_dist):
    """this takes a distribution of probabilities by rating from one to five and returns the
    expected value of the rating"""
    runsum = 0
    for i in [1, 2, 3, 4, 5]:
        runsum += rating_dist[i - 1] * i
    return runsum


In [56]:
review_dist["u.id"]

Series([], Name: u.id, dtype: object)

In [37]:
['Restaurants','Sushi Bars','Japanese','Ramen']

['5yZ1XmDcOEsElDeb9PlPDQ','PL3cimEUfNHlenOGSOAdJg','4n81G-pmC3rfhmaPsbwYKg','iwGhazq9eP51PSerTrMrwg','R3TC2oq8fQK9c9BNMZ-ynA']

,r.id,r.stars,u.id


In [ ]:
MATCH (s:State)<--(:City)<--(b:Business)-->(c:Category)
WHERE s.name="NV" and c.id in ["Japanese"," Sushi Bars","Beer", " Bars", "American (Traditional)", " Wine & Spirits", "Sports Bars", "Nightlife", "Ramen", "Pubs", " Dive Bars", "Seafood"]
RETURN count(distinct b.id)

In [100]:
def biz_preference_demo(driver, user_cat_ids, biz_id, state):

    # send a cypher query to the server that returns reviews of biz by people
    # in state
    review_dist = cypher(
        driver,
        f"MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business)-->(:City)-->(s:State)\
        WHERE b.id='{biz_id}' AND s.name='{state}'\
        RETURN r.id, r.stars, u.id LIMIT 300",
        [
            'r.id',
            'r.stars',
            'u.id'])
    
    print(review_dist.shape)
    review_stars = review_dist['r.stars'].value_counts()
    num_reviews = review_dist['r.stars'].shape[0]
    

    # we initialize a blank list of users in the user categories
    user_in_cat = []

    for cat in user_cat_ids:
        # this loop sends a crypher query to retreive users in each category in
        # the state
        temp = cypher(
            driver,
            f'MATCH (u:User)-[]->(r:Reputation)\
            WHERE r.id ="{cat}" and u.id IN {list(review_dist["u.id"])}\
            RETURN u.id LIMIT 50',
            ['u.id'])
        user_in_cat.append(temp)
        print(temp.shape)
    
        
    reviews_in_cat = []
    for i in range(len(user_in_cat)):
        # this loop goes through each user category and sends a cypher query to get the reviews of
        # the business from users in the category

        sim_user = cypher(
            driver, f'MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business)-->(:City)-->(s:State)\
            WHERE s.name= "{state}" and b.id = "{biz_id}" and u.id IN {list(user_in_cat[i]["u.id"])}\
            RETURN r.stars, u.id', ['r.stars', 'u.id'])
        reviews_in_cat.append(sim_user)
        
    
    # this loop and PRu below uses laplace smoothing and the distribution of biz reviews
    # to come up with naive bayes estimated probability distribution,
    # prob(review of biz = k)
    numerator = np.empty(5)
    for i in (1, 2, 3, 4, 5):
        try:
            numerator[i - 1] = review_stars[i]
        except BaseException:
            numerator[i - 1] = 0

    PRu = (numerator + 1) / (num_reviews + 5)

    # the code below uses laplace smoothing and the distribution of the biz reviews to come up with
    # a naive bayes estimate of the distribution (prob review of biz=k|given
    # reviewer in category j)
    num_cat = len(user_in_cat)
    cats_by_stars = np.empty((num_cat, 5))

    for i in range(num_cat):
        if not reviews_in_cat[i].empty:
            cat_stars = reviews_in_cat[i]['r.stars'].value_counts()
            for j in (1, 2, 3, 4, 5):
                try:
                    cats_by_stars[i][j - 1] = cat_stars[j]
                except BaseException:
                    cats_by_stars[i][j - 1] = 0

        else:
            # If there are no users in a category we use the review
            # distribution without the conditional

            for j in (1, 2, 3, 4, 5):
                try:
                    cats_by_stars[i][j - 1] = review_stars[j]
                except BaseException:
                    cats_by_stars[i][j - 1] = 0

    PRaj = ((cats_by_stars + 1) / (numerator + num_cat)).prod(axis=0)

    # we now take the product of the distributions and normalize them so they
    # sum to 1
    biz_prefs_un_normalized = PRu * PRaj

    biz_prefs = biz_prefs_un_normalized/sum(biz_prefs_un_normalized)

    return biz_prefs

In [36]:
pd.set_option('max_colwidth', -1)
ratingslist

,b.id,r.stars,cats
0,5yZ1XmDcOEsElDeb9PlPDQ,1,"[Restaurants, Japanese, Ramen]"
1,PL3cimEUfNHlenOGSOAdJg,1,"[Japanese, Sushi Bars, Restaurants]"
2,4n81G-pmC3rfhmaPsbwYKg,1,"[Restaurants, Food, Beer, Bars, American (Traditional), Wine & Spirits, Sports Bars, Nightlife]"
3,iwGhazq9eP51PSerTrMrwg,1,"[ Bars, Wine & Spirits, Restaurants, American (Traditional), Nightlife, Food, Pubs, Beer, Dive Bars]"
4,R3TC2oq8fQK9c9BNMZ-ynA,1,"[ Bars, Seafood, Sushi Bars, Nightlife, Restaurants, Japanese]"


In [97]:
up=user_preference_demo(driver, ratingslist, 'Os1n1_idfw9vv9kwULGJnQ')

In [101]:
bp=biz_preference_demo(driver, ['cool1','useful1','funny1'], 'Os1n1_idfw9vv9kwULGJnQ', 'NV')

(300, 3)
(38, 1)
(15, 1)
(46, 1)


In [102]:
bp,up

(array([0.22778281, 0.02881675, 0.00341197, 0.1709593 , 0.56902917]),
 array([0.74611399, 0.0634715 , 0.0634715 , 0.0634715 , 0.0634715 ]))

In [34]:
expected_rating(up),expected_rating(bp)

(1.6347150259067356, 3.8246352785126847)

In [28]:
joint_prob=up*bp/sum(up*bp)
joint_prob, expected_rating(joint_prob)

(array([0.77615773, 0.0083531 , 0.00098903, 0.0495559 , 0.16494425]),
 1.8187758442720459)

In [76]:
review_dist =pd.DataFrame({"b.id":['5yZ1XmDcOEsElDeb9PlPDQ','PL3cimEUfNHlenOGSOAdJg','4n81G-pmC3rfhmaPsbwYKg','iwGhazq9eP51PSerTrMrwg','R3TC2oq8fQK9c9BNMZ-ynA']})
review_dist["r.stars"]=[1,1,1,1,1]
review_dist

,b.id,r.stars
0,5yZ1XmDcOEsElDeb9PlPDQ,1
1,PL3cimEUfNHlenOGSOAdJg,1
2,4n81G-pmC3rfhmaPsbwYKg,1
3,iwGhazq9eP51PSerTrMrwg,1
4,R3TC2oq8fQK9c9BNMZ-ynA,1


In [95]:
def user_preference_demo(driver, review_dist, biz_id):

    # send a cypher query to the server that returns all of the biz's
    # categories
    biz_categories = cypher(driver, f"\
    MATCH (b:Business)-[:IN_CATEGORY]->(c:Category) \
    WHERE b.id='{biz_id}' RETURN c.id", ['c.id'])

    # these manipulate the biz categories and user's reviews for computation
    # later
    review_stars = review_dist['r.stars'].value_counts()
    num_reviews = review_dist['r.stars'].shape[0]
    cat_ids = list(biz_categories['c.id'])
    
    
    
    # we initialize a blank list of businesses in the biz categories
    biz_in_cat = []
    for cat in cat_ids:
        temp=[]
        for i in range(5):
            if cat in review_dist['cats'].iloc[i]:
                temp.append(review_dist['b.id'].iloc[i])
        if temp:
            biz_in_cat.append(temp)
        
    
    reviews_in_cat = []
    
    for i in range(len(biz_in_cat)):
        # this loop goes through each biz category and sends a cypher query to get the reviews of
        # businesses in that category by the user
        sim_biz = []
        
        for temp_biz in biz_in_cat[i]:
#             temp_biz=biz_in_cat[i][j]
            temp_rev=review_dist.loc[review_dist['b.id']==temp_biz]
            
            sim_biz.append(int(temp_rev['r.stars']))

        
        reviews_in_cat.append(pd.DataFrame(sim_biz, columns=['r.stars']))
    #print(reviews_in_cat)
    # this loop and PRu below uses laplace smoothing and the distribution of user's reviews
    # to come up with naive bayes estimated probability distribution,
    # prob(review from user = k)
    numerator = np.empty(5)
    for i in (1, 2, 3, 4, 5):
        try:
            numerator[i - 1] = review_stars[i]
            
        except BaseException:
            numerator[i - 1] = 0
    
    PRu = (numerator + 1) / (num_reviews + 5)

    # the code below uses laplace smoothing and the distribution of the biz reviews to come up with
    # a naive bayes estimate of the distribution (prob review from user =
    # k|given biz in category j)
    num_cat = len(biz_in_cat)
    cats_by_stars = np.empty((num_cat, 5))
    #print(reviews_in_cat)
    for i in range(num_cat):
        if not reviews_in_cat[i].empty:
            cat_stars = reviews_in_cat[i]['r.stars'].value_counts()
            #print(cat_stars)
            for j in (1, 2, 3, 4, 5):
                try:
                    cats_by_stars[i][j - 1] = cat_stars[j]
                except BaseException:
                    cats_by_stars[i][j - 1] = 0
        else:
            # If there are no businesses in a category we use the review
            # distribution without the conditional
            for j in (1, 2, 3, 4, 5):
                try:
                    cats_by_stars[i][j - 1] = 3
                except BaseException:
                    cats_by_stars[i][j - 1] = 0

    PRaj = ((cats_by_stars + 1) / (numerator + num_cat)).prod(axis=0)

    # we now take the product of the distributions and normalize them so they
    # sum to 1
    user_prefs_un_normalized = PRu * PRaj

    user_prefs = user_prefs_un_normalized / sum(user_prefs_un_normalized)

    return user_prefs


In [13]:
demo_revs_by_cat=cypher(driver, "\
MATCH (b:Business)-[:IN_CATEGORY]->(c:Category)\
WHERE b.id in ['5yZ1XmDcOEsElDeb9PlPDQ','PL3cimEUfNHlenOGSOAdJg','4n81G-pmC3rfhmaPsbwYKg','iwGhazq9eP51PSerTrMrwg','R3TC2oq8fQK9c9BNMZ-ynA']\
RETURN b.id, collect(c.id)", ['b.id', 'cats'])

demo_revs_by_cat

,b.id,cats
0,PL3cimEUfNHlenOGSOAdJg,"[Japanese, Sushi Bars, Restaurants]"
1,4n81G-pmC3rfhmaPsbwYKg,"[Restaurants, Food, Beer, Bars, American (Tra..."
2,5yZ1XmDcOEsElDeb9PlPDQ,"[Restaurants, Japanese, Ramen]"
3,iwGhazq9eP51PSerTrMrwg,"[ Bars, Wine & Spirits, Restaurants, American..."
4,R3TC2oq8fQK9c9BNMZ-ynA,"[ Bars, Seafood, Sushi Bars, Nightlife, Resta..."


In [14]:
ratingslist=review_dist.merge(demo_revs_by_cat, on='b.id')

In [64]:
for i in range(5):
    if 'Japanese' in ratingslist['cats'].iloc[i]:
        print(ratingslist['b.id'].iloc[i])
    

5yZ1XmDcOEsElDeb9PlPDQ
PL3cimEUfNHlenOGSOAdJg
R3TC2oq8fQK9c9BNMZ-ynA


In [106]:
ratingslist

,b.id,r.stars,cats
0,5yZ1XmDcOEsElDeb9PlPDQ,5,"[Restaurants, Japanese, Ramen]"
1,PL3cimEUfNHlenOGSOAdJg,5,"[Japanese, Sushi Bars, Restaurants]"
2,4n81G-pmC3rfhmaPsbwYKg,2,"[Restaurants, Food, Beer, Bars, American (Tra..."
3,iwGhazq9eP51PSerTrMrwg,2,"[ Bars, Wine & Spirits, Restaurants, American..."
4,R3TC2oq8fQK9c9BNMZ-ynA,5,"[ Bars, Seafood, Sushi Bars, Nightlife, Resta..."
